In [1]:
import numpy as np
import pandas as pd
import methods.selenium_patch as selenium_patch
from tqdm import tqdm
from methods.scraper import *
from methods.model_methods import *

In [2]:
tickers = pd.read_csv("../data/tickers/simple_tickers.csv")["Ticker"].to_list()

In [3]:
tickers[1]

'TRIFOR.CO'

In [4]:
get_raw_data("TRIFOR.CO")

,"(Fiscal Quarter, Period Ending)",Market Capitalization,Market Cap Growth,Enterprise Value,Last Close Price,PE Ratio,Forward PE,PS Ratio,PB Ratio,P/TBV Ratio,P/FCF Ratio,...,Return on Equity (ROE),Return on Assets (ROA),Return on Invested Capital (ROIC),Return on Capital Employed (ROCE),Earnings Yield,FCF Yield,Dividend Yield,Payout Ratio,Buyback Yield / Dilution,Total Shareholder Return
Current,"Feb '26 Feb 12, 2026",1663,3.60%,2293,85.50,15.90,16.33,1.02,1.64,4.93,9.50,...,4.83%,2.67%,0.99%,5.10%,6.30%,10.53%,-,-,-0.02%,-0.02%
Q3 2025,"Sep '25 Sep 30, 2025",1581,-16.96%,2251,81.60,15.10,20.31,0.97,1.56,4.69,9.03,...,0.32%,0.87%,0.27%,5.10%,6.62%,11.07%,-,-,0.23%,0.23%
Q2 2025,"Jun '25 Jun 30, 2025",1812,-15.73%,2398,93.50,-,23.79,1.13,1.81,5.66,12.39,...,2.71%,2.12%,0.84%,4.00%,5.80%,8.07%,-,-,0.92%,0.92%
Q1 2025,"Mar '25 Mar 31, 2025",1636,-28.33%,2312,84.10,14.40,20.14,1.03,1.63,5.07,10.29,...,39.92%,2.77%,1.95%,4.20%,6.94%,9.72%,-,-,-0.29%,-0.29%
Q4 2024,"Dec '24 Dec 31, 2024",1448,-29.10%,1612,74.50,11.71,27.28,0.94,1.44,4.61,13.03,...,5.36%,0.90%,0.64%,3.50%,8.54%,7.68%,-,-,-0.03%,-0.03%
Q3 2024,"Sep '24 Sep 30, 2024",1904,-17.71%,2096,98.00,14.30,19.75,1.25,2.14,9.96,22.56,...,4.38%,1.59%,1.15%,5.40%,7.00%,4.43%,-,-,0.11%,0.11%
Q2 2024,"Jun '24 Jun 30, 2024",2150,-21.01%,2314,110.80,16.48,21.32,1.40,2.40,8.66,16.45,...,7.34%,1.19%,0.74%,6.50%,6.07%,6.08%,-,297.92%,0.24%,0.24%
Q1 2024,"Mar '24 Mar 31, 2024",2282,-27.06%,2873,116.83,20.97,19.97,1.47,2.51,8.08,16.49,...,49.10%,6.46%,4.31%,7.50%,4.77%,6.07%,-,-,0.95%,0.95%
Q4 2023,"Dec '23 Dec 31, 2023",2042,-27.95%,2136,104.31,18.71,20.22,1.32,2.25,7.49,12.50,...,6.10%,2.92%,1.55%,9.20%,5.34%,8.00%,0.71%,-,0.92%,1.64%
Q3 2023,"Sep '23 Sep 30, 2023",2314,-13.54%,2480,117.82,43.01,24.45,1.53,2.93,10.66,12.55,...,-3.98%,3.95%,3.33%,9.90%,2.33%,7.97%,-,-,0.52%,0.52%


In [5]:
get_data("TRIFOR.CO")

ValueError: could not convert string to float: '-'

In [9]:
data = []
for ticker in tqdm(tickers, smoothing=0):
    data.append(get_data(ticker, frequency="quarterly"))

  1%|          | 1/83 [00:15<21:21, 15.63s/it]


ValueError: could not convert string to float: '-'

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score
import numpy as np
import copy
import time

# 1. Prepare Data
# Re-run imputer to ensure we have the dataframe
target_col = "Future Change%"

# Separate Features and Target
X_cols = df.drop(["Ticker", "Close Price", "Future Change%"], axis=1).columns.tolist()
y_col = [target_col]

# Indices mapping (0-based) - 0 is Newest
idx_pred = [0]
idx_val = [1, 2]
idx_train = list(range(3, len(df)))

# Extract numpy arrays
X = df[X_cols].values
y = df[y_col].values

# Scaling
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

# Fit only on training data
X_train_raw = X[idx_train]
y_train_raw = y[idx_train]

scaler_X.fit(X_train_raw)
scaler_y.fit(y_train_raw)

X_scaled = scaler_X.transform(X)
y_scaled = scaler_y.transform(y)

# Helper to create sliding window tensors
def create_sliding_dataset(indices, seq_len):
    xs = []
    ys = []
    
    for i in indices:
        # Check if we have enough historical data
        # We need data from index i down to i + seq_len - 1
        if i + seq_len <= len(X_scaled):
            # Extract window: starts at i (newest) goes to i+seq_len (oldest)
            # Slice gives: [i, i+1, ..., i+seq_len-1] (New -> Old)
            window = X_scaled[i : i + seq_len]
            # Reverse to get Chronological order: Old -> New
            window = window[::-1]
            
            xs.append(window)
            ys.append(y_scaled[i])
            
    if not xs:
        return None, None
        
    return torch.tensor(np.array(xs), dtype=torch.float32), torch.tensor(np.array(ys), dtype=torch.float32)

# 2. Define LSTM Model
class StockLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size, dropout_prob):
        super(StockLSTM, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout_prob)
        self.fc_1 = nn.Linear(hidden_size, 16)
        self.relu = nn.ReLU()
        self.fc_2 = nn.Linear(16, output_size)
        
    def forward(self, x):
        # x shape: (batch_size, seq_len, input_size)
        out, _ = self.lstm(x)
        out = out[:, -1, :] # Last time step (Newest data point)
        out = self.fc_1(out)
        out = self.relu(out)
        out = self.fc_2(out)
        return out

# 3. Iterate through Window Sizes
window_sizes = range(1, len(df.index))
results = {}

print(f"Comparing window sizes: {window_sizes}...")
best_window_loss = float('inf')
best_window_r2 = -float('inf')
best_window_size = -1
best_model_overall_wts = None

# Hyperparameters
HIDDEN_SIZE = 100 
NUM_LAYERS = 2

for w_size in window_sizes:
    print(f"\n--- Testing Window Size: {w_size} ---")
    
    # Create Tensors
    X_train_t, y_train_t = create_sliding_dataset(idx_train, w_size)
    X_val_t, y_val_t = create_sliding_dataset(idx_val, w_size)
    
    if X_train_t is None or X_val_t is None:
        print(f"Not enough data for window size {w_size}. Skipping.")
        continue

    # Initialize Model
    input_features = X_train_t.shape[2]
    torch.manual_seed(42)
    model = StockLSTM(input_features, hidden_size=HIDDEN_SIZE, num_layers=NUM_LAYERS, output_size=1, dropout_prob=0.3)

    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=500, factor=0.5)

    # Train
    epochs = 500 
    best_val_loss = float('inf')
    best_val_r2 = -float('inf')
    best_wts = copy.deepcopy(model.state_dict())
    
    start_time = time.time()
    
    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        outputs = model(X_train_t)
        loss = criterion(outputs, y_train_t)
        loss.backward()
        optimizer.step()
        
        # Validation
        model.eval()
        with torch.no_grad():
            val_out = model(X_val_t)
            val_loss = criterion(val_out, y_val_t)
            
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                best_val_r2 = r2_score(y_val_t.numpy(), val_out.numpy())
                best_wts = copy.deepcopy(model.state_dict())
        
        scheduler.step(val_loss)

    print(f"Window {w_size} finished. Best Val Loss: {best_val_loss:.6f}, R2: {best_val_r2:.4f}")
    results[w_size] = (best_val_loss, best_val_r2)
    
    # Check if this is the best window overall
    if best_val_loss < best_window_loss:
        best_window_loss = best_val_loss
        best_window_r2 = best_val_r2
        best_window_size = w_size
        best_model_overall_wts = best_wts

print(f"\nSearch complete.")
print(f"Best Window Size: {best_window_size} (Val Loss: {best_window_loss:.6f}, R2: {best_window_r2:.4f})")

# 4. Predict using Best Model
print("\nRetraining/Loading best model for prediction...")

# Re-construct dataset with best window to be sure indices match
X_pred_t, _ = create_sliding_dataset(idx_pred, best_window_size)
if X_pred_t is not None:
    # Re-init model structure
    input_features = X_pred_t.shape[2]
    # IMPORTANT: Must use same hidden_size as training
    model = StockLSTM(input_features, hidden_size=HIDDEN_SIZE, num_layers=NUM_LAYERS, output_size=1, dropout_prob=0.3)
    model.load_state_dict(best_model_overall_wts) # type: ignore
    model.eval()

    with torch.no_grad():
        prediction_scaled = model(X_pred_t)
        prediction = scaler_y.inverse_transform(prediction_scaled.numpy())

    print(f"Prediction for Row 0 (Newest) using Window {best_window_size}: {prediction[0][0]:.4f}")
else:
    print("Could not make a prediction due to insufficient data for the chosen window size.")

Comparing window sizes: range(1, 21)...

--- Testing Window Size: 1 ---
Window 1 finished. Best Val Loss: 0.018945, R2: -35085.5000

--- Testing Window Size: 2 ---
Window 2 finished. Best Val Loss: 0.009865, R2: -18269.6699

--- Testing Window Size: 3 ---
Window 3 finished. Best Val Loss: 0.004471, R2: -8278.9814

--- Testing Window Size: 4 ---
Window 4 finished. Best Val Loss: 0.002279, R2: -4220.7148

--- Testing Window Size: 5 ---
Window 5 finished. Best Val Loss: 0.002102, R2: -3891.3330

--- Testing Window Size: 6 ---
Window 6 finished. Best Val Loss: 0.000689, R2: -1275.8632

--- Testing Window Size: 7 ---
Window 7 finished. Best Val Loss: 0.000163, R2: -300.4445

--- Testing Window Size: 8 ---
Window 8 finished. Best Val Loss: 0.000350, R2: -647.6323

--- Testing Window Size: 9 ---
Window 9 finished. Best Val Loss: 0.000403, R2: -745.2031

--- Testing Window Size: 10 ---
Window 10 finished. Best Val Loss: 0.000350, R2: -647.6236

--- Testing Window Size: 11 ---
Window 11 finishe